# Analiza Danych - Projekt
Marcin Jeznach 241204 \
Semestr letni 2023/24

## Zbiór danych

[Statlog (German Credit Data)](https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data "Zbiór danych - UCI")

Zbiór danych zawiera dane 1000 klientów, będących kredytobiorcami banku i klasyfikuje ich na dwie klasy (dobrych kredytobiorców i złych kredytobiorców, lub inaczej niskie i wysokie ryzyko). Celem projektu jest stworzenie modelu predykcji ryzyka kredytowego.

### Opis kolumn

Zarówno w opisie zbiory danych, jak i samym zbiorze danych, kolumny nie mają opisowych nazw - opis wymienia jedynie nazwy `Attribute 1` - `Attribute 20`. Aby zwiększyć czytelność zarówno kodu, jak i tabel/wykresów, do danych przypisane zostały własne nazwy.

Wartości kolumn kategorycznych zostały zakodowane w danych źródłowych jako `A{n}{m}`, gdzie `{n}` to numer kolumny, a `{m}` to numer kategorii. Aby ponownie zwiększyć czytelność, jako słowniki dla tych kategorii, użyte zostaną opisy wartości, przetłumaczone na język polski.

Warto również zwrócić uwagę, że zbiór danych powstał w 1994 roku, przez co wartości walutowe podawane są w Markach Niemieckich (DM).

Jeżeli nie podano inaczej, to wartości czasowe są najprawdopodobniej wyrażone w latach.

| Nazwa kolumny (oryginalna) | Nazwa kolumny (własna)    | Typ kolumny  | Opis kolumny                                                |
| -------------------------- | ------------------------- | ------------ | ----------------------------------------------------------- |
| Attribute 1                | Saldo ROR                 | kategoryczna | Bieżące saldo używanego rachunku rozliczeniowego            |
| Attribute 2                | Okres kredytowania        | numeryczna   | Czas trwania kredytu, w miesiącach                          |
| Attribute 3                | Historia kredytowa        | kategoryczna | Historia dotychczasowych kredytów i terminowosci            |
| Attribute 4                | Cel konsumpcyjny          | kategoryczna | Na co planowane jest przeznaczenie pożyczonych środków      |
| Attribute 5                | Kwota kredytu             | numeryczna   |                                                             |
| Attribute 6                | Saldo oszczędności        | kategoryczna | Suma salda rachunków oszczędnościowych i wartości obligacji |
| Attribute 7                | Bieżący staż pracy        | kategoryczna | Czas pracy w bieżącym miejscu zatrudnienia                  |
| Attribute 8                | Wskaźnik RdD              | numeryczna   | Stosunek wysokości raty miesięcznej do dochodów, w %        |
| Attribute 9                | Stan cywilny              | kategoryczna |                                                             |
| Attribute 10               | Dodatkowy dłużnik         | kategoryczna | Czy do kredytu dopisane są inne osoby / w jakim charakterze |
| Attribute 11               | Bieżący czas zamieszkania | numeryczna   | Czas zamieszkania pod obecnym adresem zamieszkania          |
| Attribute 12               | Mienie                    | kategoryczna | Inne mienie nie wymienione w saldzie oszczędności           |
| Attribute 13               | Wiek                      | numeryczna   |                                                             |
| Attribute 14               | Pozostałe zobowiązania    | kategoryczna | Czy klient posiada inne zobowiązania ratalne                |
| Attribute 15               | Typ zamieszkania          | kategoryczna | Charakter w jakim klient zamieszkuje pod obecnym adresem    |
| Attribute 16               | Liczba zobowiązań         | numeryczna   | Liczba pozostałych zobowiązań                               |
| Attribute 17               | Rodzaj zawodu             | kategoryczna |                                                             |
| Attribute 18               | Osoby na utrzymaniu       | numeryczna   | Liczba osób będących na utrzymaniu klienta                  |
| Attribute 19               | Numer telefonu            | kategoryczna | Czy klient posiada / podał(a) numer telefonu                |
| Attribute 20               | Pracownik zagraniczny     | kategoryczna | Czy klient jest pracownikiem zagranicznym                   |

Dodatkowo występuje kolumna kategoryczna z oceną ryzyka kredytowego.


#### Załadowanie danych

In [28]:
import pandas as pd
import seaborn as sns

data = pd.read_csv(
	"./german.data",
	sep=" ", 
	names= [
		"Saldo ROR",
		"Okres kredytowania",
		"Historia kredytowa",
		"Cel konsumpcyjny",
		"Kwota kredytu",
		"Saldo oszczędności",
		"Bieżący staż pracy",
		"Wskaźnik RdD",
		"Stan cywilny",
		"Dodatkowy dłużnik",
		"Bieżący czas zamieszkania",
		"Mienie",
		"Wiek",
		"Pozostałe zobowiązania",
		"Typ zamieszkania",
		"Liczba zobowiązań",
		"Rodzaj zawodu",
		"Osoby na utrzymaniu",
		"Numer telefonu",
		"Pracownik zagraniczny",
		"Ocena ryzyka"
	],
	index_col= False
)

### Słowniki kolumn kategorycznych

Poniżej znajdują się mapowania znaczenia kolumn kategorycznych. Ewentualne objaśnienia znajdują się w komentarzu na końcu linii z daną wartością.

Jeżeli nie podano inaczej, to górna wartość zakresów jest wartością wyłączną (wartość `v` w zakresie `x` do `y` oznacza `x` <= `v` < `y`).

In [29]:

data["Saldo ROR"] = data["Saldo ROR"].map({
	"A11": "Poniżej 0 DM",
	"A12": "0 DM do 200 DM",
	"A13": "Ponad 200 DM",
	"A14": "Brak ROR",
})

data["Historia kredytowa"] = data["Historia kredytowa"].map({
	"A30": "Nigdy nie brano lub Wszystkie spłacono w terminach",
	"A31": "Wszystkie kredyty wewnętrzne spłacono w terminie", # Przez kredyty wewnętrzne danego banku, rozumie się takie których dany bank jest kredytodawcą
	"A32": "Istniejące kredyty dotychczasowo spłacono w terminie",
	"A33": "W przeszłości występowały opóźnienia w spłacie",
	"A34": "Krytyczny rachunek lub istniejące kredyty zewnętrzne",
})

data["Cel konsumpcyjny"] = data["Cel konsumpcyjny"].map({
	"A40": "Samochód nowy",
	"A41": "Samochód używany",
	"A42": "Meble i wyposażenie",
	"A43": "RTV",
	"A44": "AGD",
	"A45": "Naprawy",
	"A46": "Edukacja",
	"A47": "Wakacje",
	"A48": "Szkolenia", # Wartość oryginalna "retraining", może również oznaczać przekwalifikowanie
	"A49": "Wydatki biznesowe",
	"A410": "Inne",
})

data["Saldo oszczędności"] = data["Saldo oszczędności"].map({
	"A61": "Poniżej 100 DM",
	"A62": "100 DM do 500 DM",
	"A63": "500 DM do 1000 DM",
	"A64": "Powyżej 1000 DM",
	"A65": "Brak lub nieznana wartość",
})

data["Bieżący staż pracy"] = data["Bieżący staż pracy"].map({
	"A71": "Bezrobotny/a",
	"A72": "Do 1 roku",
	"A73": "1 do 4 lat",
	"A74": "4 do 7 lat",
	"A75": "Powyżej 7 lat",
})

data["Stan cywilny"] = data["Stan cywilny"].map({
	"A91": "Rozwodnik lub w separacji",
	"A92": "Rozwódka, w separacji lub zamężna",
	"A94": "Kawaler",
	"A93": "Żonaty lub wdowiec",
	"A95": "Panna",
})

data["Dodatkowy dłużnik"] = data["Dodatkowy dłużnik"].map({
	"A101": "Brak",
	"A102": "Współkredytobiorca",
	"A103": "Poręczyciel",
})

data["Mienie"] = data["Mienie"].map({
	"A121": "Nieruchomości",
	"A122": "Oszczędności w Towarzystwie Budowlanym lub Ubezpieczenie na życie",
	"A123": "Samochód lub inne (nie dotyczy oszczędności)",
	"A124": "Nieznane lub brak majątku nieruchomego",
})

data["Pozostałe zobowiązania"] = data["Pozostałe zobowiązania"].map({
	"A141": "Kredyty bankowe",
	"A142": "Zakupy na raty",
	"A143": "Brak",
})

data["Typ zamieszkania"] = data["Typ zamieszkania"].map({
	"A151": "Wynajem",
	"A152": "Nieruchomość na własność",
	"A153": "Mieszkanie socjalne", # Wartość oryginalna to "for free", ciężko znaleźć inne sensowne tłumaczenie
})

data["Rodzaj zawodu"] = data["Rodzaj zawodu"].map({
	"A171": "Bezrobotny lub niewykwalifikowany/a - brak obywatelstwa",
	"A172": "Niewykwalifikowany/a - obywatel(ka)",
	"A173": "Wykwalifikowany/a lub Urzędnik/czka",
	"A174": "Kadra menedżerska, Samozatrudnienie, Wysokie kwalifikacje lub Funkcjonariusz(ka)",
})

data["Numer telefonu"] = data["Numer telefonu"].map({
	"A191": "Brak numeru telefonu",
	"A192": "Numer telefonu zarejestrowany na danego klienta",
})

data["Pracownik zagraniczny"] = data["Pracownik zagraniczny"].map({
	"A201": "Tak",
	"A202": "Nie",
})

data["Ocena ryzyka"] = data["Ocena ryzyka"].map({
	1: "Niskie ryzyko / dobry klient",
	2: "Wysokie ryzyko / zły klient",
})